Initial Pre-Processing notebook

In [2]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from collections import Counter
from pprint import pprint
# from Bio import Entrez
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr

## GSE65682

In [30]:
# design_raw = pd.read_csv('../data/GSE65682/phenodata.txt', sep='\t')
# xml = ET.parse('../data/GSE65682/GSE65682_family.xml')
# ns = {'namespace':'http://www.ncbi.nlm.nih.gov/geo/info/MINiML'}
# platform = pd.read_csv('../data/GSE65682/GPL13667-15572.txt', sep='\t', index_col=0)
exp = pd.read_csv('../example/exp.txt', sep='\t')
# root = xml.getroot()

In [32]:
exp.rename(columns={'Unnamed: 0':'genes'}, inplace=True)
exp.head(5)

,genes,GSM1602801_03_04_2013_B02_3270.CEL,GSM1602802_03_04_2013_B06_1793.CEL,GSM1602803_03_04_2013_B07_3252.CEL,GSM1602804_03_04_2013_B08_3242.CEL,GSM1602805_03_04_2013_C08_2736.CEL,GSM1602806_03_04_2013_E06_526.CEL,GSM1602807_03_04_2013_E09_hv59.CEL,GSM1602808_03_04_2013_F02_2618.CEL,GSM1602809_03_04_2013_F10_hv49.CEL,...,GSM1692494_27_03_2013_E07_1073.CEL,GSM1692495_27_03_2013_E10_2922677.CEL,GSM1692496_27_03_2013_E11_2989698.CEL,GSM1692497_27_03_2013_F01_2968378.CEL,GSM1692498_27_03_2013_F02_2904613.CEL,GSM1692499_27_03_2013_F09_3393.CEL,GSM1692501_27_03_2013_G02_935.CEL,GSM1692502_27_03_2013_G07_499.CEL,GSM1692503_27_03_2013_G08_3316.1.CEL,GSM1692504_27_03_2013_H02_3283.CEL
0,11715100_at,2.167651,2.217742,2.167319,2.031671,2.340405,2.032680,2.213454,2.173260,2.293192,...,2.724034,3.126419,2.921860,2.579094,3.172682,2.141705,2.284317,2.338371,2.406765,2.472650
1,11715101_s_at,2.765675,2.323583,2.715749,2.657347,2.761333,2.577295,2.630158,2.653431,2.316426,...,2.651915,2.724911,2.398120,3.061678,2.249742,2.724024,2.455879,2.902131,2.820834,2.346937
2,11715102_x_at,2.375666,2.188650,2.406119,2.281674,2.122309,2.334668,2.315920,2.325296,2.319462,...,2.926233,2.875802,2.494554,2.616553,3.058945,2.338368,2.389344,2.827935,2.590062,2.639150
3,11715103_x_at,2.622201,2.131634,2.480848,2.744567,2.954394,2.617683,3.288098,2.618424,3.110793,...,2.466712,3.132406,2.795552,2.520526,3.193006,2.507180,2.439237,3.142155,2.541680,2.395692
4,11715104_s_at,2.206729,1.917760,2.032542,2.000861,2.099290,2.182521,2.272056,2.143504,1.833561,...,2.073241,3.068725,2.122303,2.055336,2.279677,2.131373,2.314353,2.308145,2.323319,2.396074


In [28]:
import json
from pprint import pprint

with open('../example/entrez_id_to_hgnc_symbol.json') as f:
    entrez_id_to_hgnc_symbol = json.load(f)
# type(entrez_id_to_hgnc_symbol)
exp['genes'] = exp['genes'].astype('str') 
exp['genes'] = exp['genes'].map(entrez_id_to_hgnc_symbol)

In [29]:
exp.head(5)

,genes,GSM1602801_03_04_2013_B02_3270.CEL,GSM1602802_03_04_2013_B06_1793.CEL,GSM1602803_03_04_2013_B07_3252.CEL,GSM1602804_03_04_2013_B08_3242.CEL,GSM1602805_03_04_2013_C08_2736.CEL,GSM1602806_03_04_2013_E06_526.CEL,GSM1602808_03_04_2013_F02_2618.CEL,GSM1602810_03_04_2013_G07_3147.CEL,GSM1602811_03_04_2013_H02_2404.CEL,...,GSM1692427_22_01_2013_B10_2739656.CEL,GSM1692429_22_01_2013_C02_2295511.CEL,GSM1692430_22_01_2013_C05_1550.CEL,GSM1692443_22_01_2013_F06_2337277.CEL,GSM1692452_22_01_2013_H08_2079564.CEL,GSM1692455_26_03_2013_A03_2792222.CEL,GSM1692482_27_03_2013_C03_2803146.CEL,GSM1692489_27_03_2013_D10_2849655.CEL,GSM1692492_27_03_2013_E04_2575353.CEL,GSM1692498_27_03_2013_F02_2904613.CEL
0,H3C8,2.226389,2.306361,2.253123,2.123169,2.434243,2.151831,2.261930,2.286426,2.212464,...,2.529478,2.372678,2.450773,2.406044,2.574771,2.422820,2.494932,2.257758,2.327790,3.252404
1,H3C8,2.761263,2.359946,2.725688,2.657941,2.832263,2.603070,2.683711,2.391393,3.139401,...,2.535944,2.114600,2.356066,2.615030,2.586535,2.881207,3.740656,2.795182,2.889858,2.166293
2,H3C8,2.432933,2.224459,2.400713,2.311345,2.126898,2.358452,2.281713,2.518797,2.481215,...,2.531517,2.311668,2.286395,2.490969,3.044778,2.165541,3.202541,2.265927,2.538030,3.099365
3,TNFAIP8L1,2.700271,2.325374,2.495166,2.773752,3.007053,2.777180,2.839187,2.731439,2.632635,...,3.310849,2.814221,2.452596,2.573080,2.607405,2.613183,2.556937,3.134644,2.838427,3.274703
4,OTOP2,2.292555,1.972904,2.071156,2.039774,2.126318,2.234433,2.171372,2.484582,2.326744,...,2.313729,2.491032,2.031278,2.269346,2.409257,1.975686,2.193042,2.064105,2.725887,2.320664


In [113]:
platform['Entrez Gene'] = [gene.split(' /// ')[0] for gene in platform['Entrez Gene']]
probe2gene_mapping = dict(zip(platform.index, platform['Entrez Gene']))

In [114]:
exp['genes'] = exp['genes'].map(probe2gene_mapping)

In [116]:
exp = exp[exp.genes != '---']

In [119]:
exp.to_csv('../data/GSE65682/exp.txt', sep='\t', index=False)

In [6]:
tags = ['pneumonia diagnoses', 'mortality_event_28days', 'time_to_event_28days']

for i in tags:
    temp_lst = []
    for char in root.findall(f"./namespace:Sample/namespace:Channel/namespace:Characteristics[@tag='{i}']", ns):
        temp_var = char.text.replace('\n','').strip()
        temp_lst.append(np.nan if temp_var == 'NA' else temp_var)
    design_raw[f'{i}'] = temp_lst

NameError: name 'root' is not defined

In [80]:
Counter(design_raw['pneumonia diagnoses'].tolist())

Counter({'no-cap': 33, 'cap': 108, nan: 577, 'hap': 84})

In [82]:
design_matrix = design_raw[design_raw['pneumonia diagnoses'].isin(('cap', 'no-cap', 'hap'))].copy()

In [83]:
from collections import Counter
Counter(design_matrix['pneumonia diagnoses'].tolist())

Counter({'no-cap': 33, 'cap': 108, 'hap': 84})

In [84]:
survivor_mapping = {np.nan:np.nan, '0':'SS', '1':'SNS'}
control_mapping = {'no-cap':'control'}
design_matrix['Target'] = design_matrix['mortality_event_28days'].map(survivor_mapping)
design_matrix['Control'] = design_matrix['pneumonia diagnoses'].map(control_mapping)

In [89]:
final_design = pd.DataFrame(data = design_matrix.FileName, columns=['FileName'])

In [90]:
final_design['Target'] = pd.concat([design_matrix['Target'].dropna(), design_matrix['Control'].dropna()]).reindex_like(design_matrix).tolist()

In [104]:
final_design = final_design[~final_design.Target.isna()]

In [105]:
final_design.reset_index(inplace=True, drop=True)

In [ ]:
final_design.head()

In [109]:
final_design.to_csv('../data/GSE65682/targets.txt', sep='\t')

## TCGA_LIHC

In [2]:
target_raw = pd.read_csv('../data/tcga_lihc/lihc_tcga_clinical_data.tsv', sep='\t')
exp_raw = pd.read_csv('../data/tcga_lihc/exp.txt', sep='\t')

In [49]:
target = pd.DataFrame()
target['FileName'] = target_raw['Sample ID']
target['Target'] = target_raw['Overall Survival Status']

In [56]:
target_nona = target[~target['Target'].isna()]

In [57]:
Counter(target_nona['Target'])

Counter({'LIVING': 247, 'DECEASED': 132})

In [58]:
for sample_name in exp_raw.columns:
    if '-11A-' in sample_name:
        new_row = {'FileName': sample_name[:15], 
                   'Target': 'Control'
                  }
        target_nona = target_nona.append(new_row, ignore_index=True)

In [59]:
Counter(target_nona['Target'])

Counter({'LIVING': 247, 'DECEASED': 132, 'Control': 50})

In [60]:
len(target_nona['FileName']), len(exp_raw.columns)

(429, 421)

In [61]:
for row in target_nona.copy().index:
    if target_nona.at[row, 'FileName'] not in [col[:15] for col in exp_raw.columns]:
        target_nona.drop(index=row, inplace=True)
len(target_nona['FileName']), len(exp_raw.columns)

(421, 421)

In [62]:
target_nona.to_csv('targets.txt', sep='\t')

## TCGA_BRCA

In [7]:
target_raw = pd.read_csv('../data/tcga_brca/brca_subtypes_matrix.txt', sep=',', index_col=0)
exp_raw = pd.read_csv('../data/tcga_brca/expression_matrix_full.txt', sep='\t')

In [17]:
target_raw['subtype'] = target_raw['subtype'].where(~target_raw['subtype'].isna(), "Control")
target_raw.drop(columns='tumor', inplace=True)

In [22]:
for row in target_raw.index:
    if target_raw.at[row, 'barcode'] not in exp_raw.columns:
        target_raw.drop(index=row, inplace=True)
len(target_raw), len(exp_raw.columns)

(1215, 1215)

In [27]:
target_raw.columns = ['FileName', 'Target']
target = target_raw.reset_index(drop=True)

In [29]:
target.to_csv('../data/tcga_brca/targets.txt', sep='\t')

## HIPPIE Data

In [5]:
kg_raw = pd.read_csv('../data/kg/HIPPIE-current.mitab.txt', sep='\t')

In [6]:
kg = pd.DataFrame()
kg['Source'] = kg_raw['Gene Name Interactor A']
kg['Conf_Val'] = kg_raw['Confidence Value']
kg['Target'] = kg_raw['Gene Name Interactor B']

In [7]:
kg.to_csv('../data/kg/hippie.tsv', sep='\t', index=False)